### Check if configuration has been run already and change directory to repository root:

In [1]:
from scripts.conf_file_finding import try_find_conf_file
try_find_conf_file()

Local configuration file found !!, no need to run the configuration (unless configuration has changed)


In [2]:
import datajoint as dj
import os
import getpass
import pandas as pd
import subprocess
import json
import utility.dj_shortcuts as dj_short
dj.__version__

'0.13.2'

### Declare AutoPipeline and Status tables if they don't exist

In [3]:
from bl_pipeline import acquisition
acquisition.StatusDefinition
acquisition.AcquisitionSessionsTestAutoPipeline
acquisition.AcquisitionSessionsStatus


Connecting alvaros@datajoint01.pni.princeton.edu:3306


bl_pipeline.acquisition.AcquisitionSessionsStatus

In [4]:
## Just for now, folders for ephys things in Globus
#FOR PNI endpoint
pni_ep_id = '6ce834d6-ff8a-11e6-bad1-22000b9a448b'
pni_ephys_root_data_dir   = '/jukebox/archive/brody/RATTER/PhysData/Raw/'
pni_ephys_sorted_data_dir = '/mnt/cup/labs/brody/RATTER/PhysData/Test_ephys_pipeline_NP_sorted/'

#For tiger endpoint
tiger_ep_dir = 'a9df83d2-42f0-11e6-80cf-22000b1701d1'
tiger_ephys_root_data_dir = '/tigress/alvaros/ephys_raw/'
tiger_ephys_sorted_data_dir = '/tigress/alvaros/ephys_sorted/'


### Delete and insert on the AcquisitionSessionsTestAutoPipeline and Status tables

In [ ]:
acquisition.AcquisitionSessionsTestAutoPipeline.delete()
acquisition.AcquisitionSessionsStatus.delete

In [ ]:
fields_t1 = pd.DataFrame.from_dict(acquisition.AcquisitionSessionsTestAutoPipeline.heading.attributes, orient='index')
fields_t1_list = fields_t1.loc[fields_t1['in_key'] == True].index.to_list()
fields_t1_list

### Get 5 "real" sessions to test and insert them to the TestAutoPipeline

In [ ]:
acq_sessions_df = pd.DataFrame(acquisition.AcquisitionSessions.fetch(limit=5, as_dict=True))
acq_sessions_df['status_pipeline'] = 0
acquisition.AcquisitionSessionsTestAutoPipeline.insert(acq_sessions_df, skip_duplicates=True)
acquisition.AcquisitionSessionsTestAutoPipeline()

### Check if logged to globus

In [ ]:
s = subprocess.run(["globus", "whoami"], capture_output=True)
s.stdout.decode('UTF-8')

### Get all "active sessions"

In [21]:
sessions_active = acquisition.AcquisitionSessionsTestAutoPipeline & 'status_pipeline < 7 and status_pipeline >= 0'
df_sessions = pd.DataFrame(sessions_active.fetch(as_dict=True))

#Add a column to have a key dictionary as a new column
key_list = dj_short.get_primary_key_fields(acquisition.AcquisitionSessionsTestAutoPipeline)
df_sessions['query_key'] = df_sessions.loc[:, key_list].to_dict(orient='records')
df_sessions

,sessid,status_pipeline,session_rat,session_userid,session_rigid,acquisition_type,acquisition_raw_rel_path,acquisition_post_rel_path,task_copy_id_pre_path,task_copy_id_pos_path,slurm_id_sorting,query_key
0,585444,1,T170,zhihaol,202,ephys,Thomas/T170/T170_2018_03_16,Thomas/T170/T170_2018_03_16_600s/spikesort_202...,fc167e0e-3277-11ec-9e4f-3df4ed83d858,None,None,{'sessid': 585444}
1,585488,0,T173,zhihaol,202,ephys,Thomas/T173/T173_bank1_2018_03_16,Thomas/T173/T173_2018_03_16_bank1_600s/spikeso...,None,None,None,{'sessid': 585488}
2,586335,0,T170,zhihaol,202,ephys,Thomas/T170/T170_2018_03_19,Thomas/T170/T170_2018_03_19/spikesort_2020_09_...,None,None,None,{'sessid': 586335}
3,586376,0,T173,zhihaol,202,ephys,Thomas/T173/T173_bank0_2018_03_19,Thomas/T173/T173_2018_03_19_bank0_600s/spikeso...,None,None,None,{'sessid': 586376}
4,590577,0,T170,zhihaol,202,ephys,Thomas/T170/T170_2018_04_05,Thomas/T170/T170_2018_04_05/spikesort_2020_09_...,None,None,None,{'sessid': 590577}


In [17]:
session_start_df = df_sessions.loc[df_sessions['status_pipeline'] == 0, :]
session_start_df = session_start_df.reset_index(drop=True)
session_start_df

,sessid,status_pipeline,session_rat,session_userid,session_rigid,acquisition_type,acquisition_raw_rel_path,acquisition_post_rel_path,task_copy_id_pre_path,task_copy_id_pos_path,slurm_id_sorting,query_key
0,585444,0,T170,zhihaol,202,ephys,Thomas/T170/T170_2018_03_16,Thomas/T170/T170_2018_03_16_600s/spikesort_202...,None,None,None,{'sessid': 585444}
1,585488,0,T173,zhihaol,202,ephys,Thomas/T173/T173_bank1_2018_03_16,Thomas/T173/T173_2018_03_16_bank1_600s/spikeso...,None,None,None,{'sessid': 585488}
2,586335,0,T170,zhihaol,202,ephys,Thomas/T170/T170_2018_03_19,Thomas/T170/T170_2018_03_19/spikesort_2020_09_...,None,None,None,{'sessid': 586335}
3,586376,0,T173,zhihaol,202,ephys,Thomas/T173/T173_bank0_2018_03_19,Thomas/T173/T173_2018_03_19_bank0_600s/spikeso...,None,None,None,{'sessid': 586376}
4,590577,0,T170,zhihaol,202,ephys,Thomas/T170/T170_2018_04_05,Thomas/T170/T170_2018_04_05/spikesort_2020_09_...,None,None,None,{'sessid': 590577}


In [20]:
#session_start_df.shape[0]
for i in range(1):
    #raw_rel_path = session_start_df.loc[i, 'acquisition_raw_rel_path']
    raw_rel_path = 'Ahmed/SpikeGadgets'
    source_ep = pni_ep_id    +':'+ pni_ephys_root_data_dir   + raw_rel_path
    dest_ep   = tiger_ep_dir +':'+ tiger_ephys_root_data_dir + raw_rel_path
    globus_command = ["globus", "transfer", source_ep, dest_ep, '--recursive', '--format', 'json']
    print(globus_command)
    s = subprocess.run(globus_command, capture_output=True)
    transfer_request = json.loads(s.stdout.decode('UTF-8'))
    if transfer_request['code'] == 'Accepted':
        task_id = transfer_request['task_id']
        dict_query = session_start_df.loc[i, 'query_key']
        dict_update_task_id = dict_query.copy()
        dict_update_task_id['task_copy_id_pre_path'] = task_id
        dict_update_status = dict_query.copy()
        dict_update_status['status_pipeline'] = 1
        print(dict_update_task_id)
        print(dict_update_status)
        acquisition.AcquisitionSessionsTestAutoPipeline.update1(dict_update_task_id)
        acquisition.AcquisitionSessionsTestAutoPipeline.update1(dict_update_status)
        
    

['globus', 'transfer', '6ce834d6-ff8a-11e6-bad1-22000b9a448b:/jukebox/archive/brody/RATTER/PhysData/Raw/Ahmed/SpikeGadgets', 'a9df83d2-42f0-11e6-80cf-22000b1701d1:/tigress/alvaros/ephys_raw/Ahmed/SpikeGadgets', '--recursive', '--format', 'json']
{'sessid': 585444, 'task_copy_id_pre_path': 'fc167e0e-3277-11ec-9e4f-3df4ed83d858'}
{'sessid': 585444, 'status_pipeline': 1}


In [22]:
session_stat_1 = df_sessions.loc[df_sessions['status_pipeline'] == 1, :]
session_stat_1 = session_stat_1.reset_index(drop=True)
session_stat_1

,sessid,status_pipeline,session_rat,session_userid,session_rigid,acquisition_type,acquisition_raw_rel_path,acquisition_post_rel_path,task_copy_id_pre_path,task_copy_id_pos_path,slurm_id_sorting,query_key
0,585444,1,T170,zhihaol,202,ephys,Thomas/T170/T170_2018_03_16,Thomas/T170/T170_2018_03_16_600s/spikesort_202...,fc167e0e-3277-11ec-9e4f-3df4ed83d858,None,None,{'sessid': 585444}


In [29]:
#session_start_df.shape[0]
for i in range(1):
    
    id_task = session_stat_1.loc[i, 'task_copy_id_pre_path']
    raw_rel_path = 'Ahmed/SpikeGadgets'
    source_ep = pni_ep_id    +':'+ pni_ephys_root_data_dir   + raw_rel_path
    dest_ep   = tiger_ep_dir +':'+ tiger_ephys_root_data_dir + raw_rel_path
    globus_command = ["globus", "task", "show", str(id_task), '--format', 'json']
    print(globus_command)
    s = subprocess.run(globus_command, capture_output=True)
    transfer_request = json.loads(s.stdout.decode('UTF-8'))
    if transfer_request['status'] == 'SUCCEEDED':
        dict_query = session_start_df.loc[i, 'query_key']
        dict_update_status = dict_query.copy()
        dict_update_status['status_pipeline'] = 2
        acquisition.AcquisitionSessionsTestAutoPipeline.update1(dict_update_status)
        
request_dir

['globus', 'task', 'show', 'fc167e0e-3277-11ec-9e4f-3df4ed83d858', '--format', 'json']


{'bytes_checksummed': 0,
 'bytes_transferred': 0,
 'canceled_by_admin': None,
 'canceled_by_admin_message': None,
 'command': 'API 0.10',
 'completion_time': '2021-10-21T14:05:48+00:00',
 'deadline': '2021-10-22T14:05:46+00:00',
 'delete_destination_extra': False,
 'destination_endpoint': 'princeton#tigress-dtn',
 'destination_endpoint_display_name': 'Princeton TIGRESS',
 'destination_endpoint_id': 'a9df83d2-42f0-11e6-80cf-22000b1701d1',
 'directories': 1,
 'effective_bytes_per_second': 0,
 'encrypt_data': True,
 'fail_on_quota_errors': False,
 'fatal_error': None,
 'faults': 0,
 'files': 0,
 'files_skipped': 0,
 'files_transferred': 0,
 'filter_rules': None,
 'history_deleted': False,
 'is_ok': None,
 'is_paused': False,
 'label': None,
 'nice_status': None,
 'nice_status_details': None,
 'nice_status_expires_in': None,
 'nice_status_short_description': None,
 'owner_id': 'a05c014a-be74-4701-9db3-abbb57b57f8c',
 'preserve_timestamp': False,
 'recursive_symlinks': 'ignore',
 'request_t

In [30]:
sessions_active = acquisition.AcquisitionSessionsTestAutoPipeline & 'status_pipeline < 7 and status_pipeline >= 0'
df_sessions = pd.DataFrame(sessions_active.fetch(as_dict=True))
df_sessions

,sessid,status_pipeline,session_rat,session_userid,session_rigid,acquisition_type,acquisition_raw_rel_path,acquisition_post_rel_path,task_copy_id_pre_path,task_copy_id_pos_path,slurm_id_sorting
0,585444,2,T170,zhihaol,202,ephys,Thomas/T170/T170_2018_03_16,Thomas/T170/T170_2018_03_16_600s/spikesort_202...,fc167e0e-3277-11ec-9e4f-3df4ed83d858,None,None
1,585488,0,T173,zhihaol,202,ephys,Thomas/T173/T173_bank1_2018_03_16,Thomas/T173/T173_2018_03_16_bank1_600s/spikeso...,None,None,None
2,586335,0,T170,zhihaol,202,ephys,Thomas/T170/T170_2018_03_19,Thomas/T170/T170_2018_03_19/spikesort_2020_09_...,None,None,None
3,586376,0,T173,zhihaol,202,ephys,Thomas/T173/T173_bank0_2018_03_19,Thomas/T173/T173_2018_03_19_bank0_600s/spikeso...,None,None,None
4,590577,0,T170,zhihaol,202,ephys,Thomas/T170/T170_2018_04_05,Thomas/T170/T170_2018_04_05/spikesort_2020_09_...,None,None,None


In [ ]:
meso = json.loads(meesage)
meso

In [ ]:
s.stdout.decode('UTF-8')

In [ ]:
so ='Message: The transfer has been accepted and a task has been created and queued for execution\nTask ID: 48e1405e-31f4-11ec-95e6-853490a236f9\n'
so

In [ ]:
s.stderr